# Corporación Favorita Grocery Sales Forecasting
**w01_d03_EDA_quality_preprocessing.ipynb**

**Author:** Alberto Diaz Durana  
**Date:** November 2025  
**Purpose:** Clean data, detect outliers with 3 methods, analyze store performance, understand item coverage

---

## Objectives

This notebook accomplishes the following:

- Load guayas_sample_300k.pkl and validate characteristics
- Handle missing values (onpromotion → fill with False)
- Detect outliers using 3 methods: IQR, Z-score, Isolation Forest
- Analyze store-level performance (11 stores comparison)
- Create item coverage matrix (product availability across stores)
- Fill calendar gaps for complete daily time series
- Extract date features (year, month, day, day_of_week)

---

## Business Context

**Why data quality matters:**

Clean, well-understood data is critical for reliable forecasting. Before modeling, we must:
- Eliminate data quality issues (missing values, outliers, negative sales)
- Understand store performance patterns (identify top/bottom performers)
- Map product availability (which items sell where)
- Ensure complete temporal coverage (no date gaps)

**Three-method outlier detection:**
- **IQR:** Robust, non-parametric (1.5×IQR rule)
- **Z-score:** Statistical, parametric (|z| > 3.0)
- **Isolation Forest:** ML-based, multivariate patterns
- **Triangulation:** High confidence when all 3 methods agree

**Deliverables:**
- Clean dataset (no missing values, no negatives, outliers flagged)
- Store performance report (sales by store, type, city, cluster)
- Item coverage matrix (2,296 items × 11 stores)
- Complete daily calendar (no date gaps)
- Date features for temporal analysis

---

## Input Dependencies

From Day 2:
- guayas_sample_300k.pkl (300K rows, 11 stores, 2,296 items)
- stores.csv (store metadata)
- items.csv (product metadata)

---

## 1. Setup & Load Data

**Objective:** Import libraries, load sample dataset, validate characteristics

**Activities:**
- Import pandas, numpy, matplotlib, seaborn, scikit-learn
- Configure environment and paths
- Load guayas_sample_300k.pkl
- Display shape, dtypes, memory usage
- Show first/last rows
- Summary statistics

**Expected output:** Dataset loaded, initial validation complete

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from sklearn.ensemble import IsolationForest

# Configure environment
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
plt.style.use('seaborn-v0_8-darkgrid')

print("Package Versions:")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")
print(f"  scikit-learn: {IsolationForest.__module__.split('.')[0]}")
print("\nOK - Libraries imported")

In [ ]:
# Determine paths
current_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
project_root = current_dir.parent if current_dir.name == 'notebooks' else current_dir

# Define path constants
DATA_PROCESSED = project_root / 'data' / 'processed'
DATA_RAW = project_root / 'data' / 'raw'
OUTPUTS = project_root / 'outputs' / 'figures' / 'eda'

# Verify paths
assert DATA_PROCESSED.exists(), f"ERROR - Path not found: {DATA_PROCESSED}"
assert DATA_RAW.exists(), f"ERROR - Path not found: {DATA_RAW}"

print("OK - Paths validated:")
print(f"  Project root: {project_root.resolve()}")
print(f"  DATA_PROCESSED: {DATA_PROCESSED.resolve()}")
print(f"  OUTPUTS: {OUTPUTS.resolve()}")

# Set random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
print(f"\nRandom seed: {RANDOM_SEED}")

In [ ]:
# Load sample dataset
print("Loading guayas_sample_300k.pkl...")
df = pd.read_pickle(DATA_PROCESSED / 'guayas_sample_300k.pkl')

print(f"OK - Dataset loaded")
print(f"\nShape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print("\nColumns:")
print(df.dtypes)

print("\nFirst 5 rows:")
print(df.head())

print("\nLast 5 rows:")
print(df.tail())

In [ ]:
# Summary statistics
print("Summary Statistics:")
print(df.describe())

print("\nMissing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

print("\nValue Counts:")
print(f"\nUnique stores: {df['store_nbr'].nunique()}")
print(f"Unique items: {df['item_nbr'].nunique()}")
print(f"Unique families: {df['family'].nunique()}")
print(f"\nFamily distribution:")
print(df['family'].value_counts())

print("\nDate range:")
print(f"  First date: {df['date'].min()}")
print(f"  Last date: {df['date'].max()}")

print("\nNegative sales check:")
negative_count = (df['unit_sales'] < 0).sum()
print(f"  Negative unit_sales: {negative_count} rows ({negative_count/len(df)*100:.3f}%)")
if negative_count > 0:
    print(f"  Min value: {df['unit_sales'].min():.2f}")

## 2. Missing Value Analysis

**Objective:** Handle missing values in onpromotion column

**Activities:**
- Visualize missing pattern (if useful)
- Fill onpromotion NaN with 0.0 (False - assume no promotion)
- Validate no missing values remain
- Document decision in decision log

**Expected output:** 
- Clean dataset with no missing values
- Decision documented (DEC-003)

In [ ]:
# Analyze onpromotion missing pattern
print("onpromotion Missing Value Analysis:")
print(f"  Total missing: {df['onpromotion'].isnull().sum():,} ({df['onpromotion'].isnull().sum()/len(df)*100:.2f}%)")

print("\nonpromotion value distribution (before filling):")
print(df['onpromotion'].value_counts(dropna=False))

print("\nMissing by family:")
missing_by_family = df.groupby('family')['onpromotion'].apply(lambda x: x.isnull().sum())
print(missing_by_family)

print("\nMissing by year:")
df['year_temp'] = pd.to_datetime(df['date']).dt.year
missing_by_year = df.groupby('year_temp')['onpromotion'].apply(lambda x: x.isnull().sum())
print(missing_by_year)
df.drop('year_temp', axis=1, inplace=True)

In [ ]:
# Fill missing onpromotion with 0.0 (False - assume no promotion)
print("Filling onpromotion NaN with 0.0 (assume no promotion)...")
print(f"Before: {df['onpromotion'].isnull().sum()} missing values")

df['onpromotion'] = df['onpromotion'].fillna(0.0)

print(f"After: {df['onpromotion'].isnull().sum()} missing values")

print("\nonpromotion value distribution (after filling):")
print(df['onpromotion'].value_counts())

print("\nPromotion rate:")
promo_rate = (df['onpromotion'] == 1.0).sum() / len(df) * 100
print(f"  Items on promotion: {(df['onpromotion'] == 1.0).sum():,} ({promo_rate:.2f}%)")
print(f"  Items not on promotion: {(df['onpromotion'] == 0.0).sum():,} ({100-promo_rate:.2f}%)")

print("\nValidate no missing values in dataset:")
print(df.isnull().sum().sum())
print("OK - No missing values remain" if df.isnull().sum().sum() == 0 else "WARNING - Missing values still present")

## 3. Outlier Detection - Three Methods

**Objective:** Detect outliers using IQR, Z-score, and Isolation Forest; handle negative sales

**Activities:**
- Handle negative sales (13 rows → clip to 0)
- IQR method: 1.5×IQR rule per store-item group
- Z-score method: |z| > 3.0 per store-item group
- Isolation Forest: ML-based multivariate detection
- Compare methods with Venn diagram
- Identify high-confidence outliers (all 3 agree)

**Expected output:** 
- Cleaned unit_sales (no negatives)
- Outlier flags from 3 methods
- Method comparison visualization
- Decision log entry (DEC-004)

In [ ]:
# Step 1: Handle negative sales (returns)
print("Step 1: Handling Negative Sales (Returns)")
print("=" * 60)

negative_mask = df['unit_sales'] < 0
negative_count = negative_mask.sum()

print(f"Negative sales found: {negative_count} rows ({negative_count/len(df)*100:.4f}%)")
print("Note: Negative values represent product returns")

if negative_count > 0:
    print(f"  Min value: {df['unit_sales'].min():.2f}")
    print(f"\n  Negative sales details:")
    print(df[negative_mask][['date', 'store_nbr', 'item_nbr', 'unit_sales', 'family']].to_string())
    
    # Clip to 0 (business decision: forecast demand, not net sales)
    df.loc[negative_mask, 'unit_sales'] = 0.0
    print(f"\nBusiness Decision: Clipped {negative_count} returns to 0")
    print("Rationale: Forecasting demand (purchases), not net sales (purchases - returns)")
    print(f"New min value: {df['unit_sales'].min():.2f}")
else:
    print("  No negative values found")

In [ ]:
# Step 2: IQR Method (Interquartile Range)
print("\nStep 2: IQR Method - Robust Outlier Detection")
print("=" * 60)

# Calculate Q1, Q3, IQR per store-item group
Q1 = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform('quantile', 0.25)
Q3 = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform('quantile', 0.75)
IQR = Q3 - Q1

# Flag outliers using 1.5×IQR rule
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df['outlier_iqr'] = (df['unit_sales'] < lower_bound) | (df['unit_sales'] > upper_bound)

iqr_count = df['outlier_iqr'].sum()
print(f"IQR outliers detected: {iqr_count:,} ({iqr_count/len(df)*100:.2f}%)")

print(f"\nIQR Statistics:")
print(f"  Mean IQR: {IQR.mean():.2f}")
print(f"  Median IQR: {IQR.median():.2f}")
print(f"  Mean upper bound: {upper_bound.mean():.2f}")

print(f"\nSample outliers (IQR method):")
print(df[df['outlier_iqr']][['date', 'store_nbr', 'item_nbr', 'unit_sales', 'family']].head(10))

In [ ]:
# Step 3: Z-Score Method (Statistical)
print("\nStep 3: Z-Score Method - Statistical Outlier Detection")
print("=" * 60)

# Calculate z-scores per store-item group
z_scores = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform(
    lambda x: (x - x.mean()) / x.std() if x.std() > 0 else 0
)

# Flag extreme outliers (|z| > 3.0)
df['outlier_zscore'] = abs(z_scores) > 3.0

zscore_count = df['outlier_zscore'].sum()
print(f"Z-score outliers detected: {zscore_count:,} ({zscore_count/len(df)*100:.2f}%)")

print(f"\nZ-Score Statistics:")
print(f"  Mean |z|: {abs(z_scores).mean():.2f}")
print(f"  Max |z|: {abs(z_scores).max():.2f}")
print(f"  % with |z| > 2: {(abs(z_scores) > 2).sum() / len(df) * 100:.2f}%")
print(f"  % with |z| > 3: {(abs(z_scores) > 3).sum() / len(df) * 100:.2f}%")

print(f"\nSample outliers (Z-score method):")
df_temp = df.copy()
df_temp['z_score'] = z_scores
print(df_temp[df_temp['outlier_zscore']][['date', 'store_nbr', 'item_nbr', 'unit_sales', 'z_score', 'family']].head(10))
df_temp.drop('z_score', axis=1, inplace=True)

In [ ]:
# Step 4: Isolation Forest Method (ML-based multivariate)
print("\nStep 4: Isolation Forest - ML-Based Multivariate Detection")
print("=" * 60)

# Prepare features for Isolation Forest
# Convert date to numeric for ML model
df['date_numeric'] = pd.to_datetime(df['date']).astype('int64') // 10**9  # Unix timestamp
df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek
df['month'] = pd.to_datetime(df['date']).dt.month

# Select features for anomaly detection
features_for_iso = ['store_nbr', 'item_nbr', 'unit_sales', 'day_of_week', 'month', 'onpromotion']
X = df[features_for_iso].copy()

print(f"Training Isolation Forest on {len(X):,} samples...")
print(f"Features: {features_for_iso}")

# Train Isolation Forest
iso_forest = IsolationForest(
    contamination=0.05,  # Expect 5% anomalies
    random_state=RANDOM_SEED,
    n_estimators=100,
    max_samples='auto',
    verbose=0
)

# Predict anomalies (-1 = outlier, 1 = inlier)
predictions = iso_forest.fit_predict(X)
df['outlier_forest'] = (predictions == -1)

forest_count = df['outlier_forest'].sum()
print(f"Isolation Forest outliers detected: {forest_count:,} ({forest_count/len(df)*100:.2f}%)")

print(f"\nSample outliers (Isolation Forest method):")
print(df[df['outlier_forest']][['date', 'store_nbr', 'item_nbr', 'unit_sales', 'family']].head(10))

# Clean up temporary columns
df.drop(['date_numeric'], axis=1, inplace=True)

In [ ]:
# Step 5: Compare Three Methods
print("\nStep 5: Method Comparison & Consensus")
print("=" * 60)

# Count outliers per method
print("Outlier counts by method:")
print(f"  IQR:              {df['outlier_iqr'].sum():>7,} ({df['outlier_iqr'].sum()/len(df)*100:>5.2f}%)")
print(f"  Z-score:          {df['outlier_zscore'].sum():>7,} ({df['outlier_zscore'].sum()/len(df)*100:>5.2f}%)")
print(f"  Isolation Forest: {df['outlier_forest'].sum():>7,} ({df['outlier_forest'].sum()/len(df)*100:>5.2f}%)")

# Consensus analysis
df['outlier_any'] = df['outlier_iqr'] | df['outlier_zscore'] | df['outlier_forest']
df['outlier_2plus'] = (df['outlier_iqr'].astype(int) + 
                        df['outlier_zscore'].astype(int) + 
                        df['outlier_forest'].astype(int)) >= 2
df['outlier_all3'] = df['outlier_iqr'] & df['outlier_zscore'] & df['outlier_forest']

print("\nConsensus analysis:")
print(f"  Flagged by any method:        {df['outlier_any'].sum():>7,} ({df['outlier_any'].sum()/len(df)*100:>5.2f}%)")
print(f"  Flagged by 2+ methods:        {df['outlier_2plus'].sum():>7,} ({df['outlier_2plus'].sum()/len(df)*100:>5.2f}%)")
print(f"  Flagged by all 3 (consensus): {df['outlier_all3'].sum():>7,} ({df['outlier_all3'].sum()/len(df)*100:>5.2f}%)")

print("\nMethod overlap:")
print(f"  IQR only:        {(df['outlier_iqr'] & ~df['outlier_zscore'] & ~df['outlier_forest']).sum():>7,}")
print(f"  Z-score only:    {(df['outlier_zscore'] & ~df['outlier_iqr'] & ~df['outlier_forest']).sum():>7,}")
print(f"  Forest only:     {(df['outlier_forest'] & ~df['outlier_iqr'] & ~df['outlier_zscore']).sum():>7,}")
print(f"  IQR + Z-score:   {(df['outlier_iqr'] & df['outlier_zscore'] & ~df['outlier_forest']).sum():>7,}")
print(f"  IQR + Forest:    {(df['outlier_iqr'] & df['outlier_forest'] & ~df['outlier_zscore']).sum():>7,}")
print(f"  Z-score + Forest:{(df['outlier_zscore'] & df['outlier_forest'] & ~df['outlier_iqr']).sum():>7,}")
print(f"  All 3 methods:   {df['outlier_all3'].sum():>7,}")

print("\nHigh-confidence outliers (all 3 methods agree):")
if df['outlier_all3'].sum() > 0:
    print(df[df['outlier_all3']][['date', 'store_nbr', 'item_nbr', 'unit_sales', 'family']].head(20))
else:
    print("  No outliers flagged by all 3 methods")

In [ ]:
# Visualize method comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Outlier counts by method
ax1 = axes[0]
methods = ['IQR', 'Z-score', 'Forest', 'Any', '2+ methods', 'All 3']
counts = [
    df['outlier_iqr'].sum(),
    df['outlier_zscore'].sum(),
    df['outlier_forest'].sum(),
    df['outlier_any'].sum(),
    df['outlier_2plus'].sum(),
    df['outlier_all3'].sum()
]
colors = ['#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
bars = ax1.bar(methods, counts, color=colors, alpha=0.7, edgecolor='black')
ax1.set_ylabel('Outlier Count', fontsize=12)
ax1.set_title('Outlier Detection - Method Comparison', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Add count labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}\n({height/len(df)*100:.2f}%)',
             ha='center', va='bottom', fontsize=9)

# Plot 2: Distribution comparison (outliers vs non-outliers)
ax2 = axes[1]
ax2.boxplot([
    df[~df['outlier_any']]['unit_sales'],
    df[df['outlier_2plus']]['unit_sales'],
    df[df['outlier_all3']]['unit_sales']
], labels=['Normal', '2+ methods', 'All 3 methods'], showfliers=False)
ax2.set_ylabel('Unit Sales', fontsize=12)
ax2.set_title('Sales Distribution by Outlier Consensus', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUTS / '01_outlier_detection_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nOK - Visualization saved to outputs/figures/eda/01_outlier_detection_comparison.png")

In [ ]:
# Summary and decision documentation
print("=" * 70)
print("OUTLIER DETECTION SUMMARY")
print("=" * 70)

print("\nThree-Method Triangulation Results:")
print(f"  Total samples: {len(df):,}")
print(f"  High-confidence outliers (all 3 agree): {df['outlier_all3'].sum():,} ({df['outlier_all3'].sum()/len(df)*100:.2f}%)")
print(f"  Moderate confidence (2+ methods): {df['outlier_2plus'].sum():,} ({df['outlier_2plus'].sum()/len(df)*100:.2f}%)")
print(f"  Low confidence (1 method only): {(df['outlier_any'].sum() - df['outlier_2plus'].sum()):,}")

print("\nBusiness Interpretation:")
print("  - 0.28% extreme outliers (all 3 methods) likely represent:")
print("    • Promotional spikes (holiday periods)")
print("    • Bulk purchases (institutional buyers)")
print("    • Data entry errors (requires investigation)")
print("  - 1.65% moderate outliers (2+ methods) represent:")
print("    • Strong promotional effects")
print("    • Seasonal demand peaks")
print("  - Decision: FLAG but do NOT remove outliers")
print("    • Sales spikes are real business events")
print("    • Model should learn these patterns")
print("    • Flag for separate analysis if needed")

print("\nOutlier flags added to dataset:")
print("  - outlier_iqr: IQR method")
print("  - outlier_zscore: Z-score method")
print("  - outlier_forest: Isolation Forest")
print("  - outlier_all3: High-confidence consensus")

print("\nDECISION (DEC-004): Three-Method Outlier Detection")
print("  Context: Need robust outlier identification for sales data")
print("  Methods: IQR (robust) + Z-score (statistical) + Isolation Forest (ML)")
print("  Decision: Flag outliers but retain in dataset")
print("  Rationale: Sales spikes are legitimate business events (promotions, holidays)")
print("  Impact: 846 high-confidence outliers (0.28%) flagged for investigation")

print("\n" + "=" * 70)
print("SECTION 3 COMPLETE - Outlier Detection")
print("=" * 70)

## 4. Store-Level Performance Analysis

**Objective:** Compare sales performance across 11 Guayas stores, analyze by type, city, and cluster

**Activities:**
- Total sales by store (identify top/bottom performers)
- Sales by store type (A/B/C/D/E comparison)
- Sales by city (Guayaquil vs Daule/Playas/Libertad)
- Sales by cluster (1, 3, 6, 10, 17)
- Visualize performance patterns

**Expected output:** 
- Store performance ranking
- Type/city/cluster analysis
- Visualizations for stakeholder communication

In [ ]:
# Load store metadata for analysis
print("Loading store metadata...")
df_stores = pd.read_csv(DATA_RAW / 'stores.csv')

# Merge with main dataset
df = df.merge(df_stores[['store_nbr', 'city', 'state', 'type', 'cluster']], 
              on='store_nbr', how='left')

print(f"OK - Merged with stores.csv")
print(f"New columns: {['city', 'state', 'type', 'cluster']}")
print(f"\nStore distribution in sample:")
print(df['store_nbr'].value_counts().sort_index())

In [ ]:
# Analyze total sales by store
print("Store Performance Analysis")
print("=" * 70)

store_performance = df.groupby('store_nbr').agg({
    'unit_sales': ['sum', 'mean', 'std', 'count']
}).round(2)
store_performance.columns = ['Total Sales', 'Avg Sales', 'Std Dev', 'Sample Count']
store_performance = store_performance.sort_values('Total Sales', ascending=False)

print("\nTop/Bottom 5 Stores by Total Sales:")
print(store_performance)

# Add store metadata
store_performance = store_performance.merge(
    df_stores[['store_nbr', 'city', 'type', 'cluster']],
    left_index=True,
    right_on='store_nbr'
).set_index('store_nbr')

print("\nStore Performance with Metadata:")
print(store_performance)

print(f"\nPerformance spread:")
print(f"  Top store (#{store_performance.index[0]}): {store_performance.iloc[0]['Total Sales']:,.0f} total sales")
print(f"  Bottom store (#{store_performance.index[-1]}): {store_performance.iloc[-1]['Total Sales']:,.0f} total sales")
print(f"  Ratio (Top/Bottom): {store_performance.iloc[0]['Total Sales'] / store_performance.iloc[-1]['Total Sales']:.2f}x")

In [ ]:
# Analyze by store type
print("\nPerformance by Store Type:")
print("=" * 70)

type_performance = df.groupby('type').agg({
    'unit_sales': ['sum', 'mean', 'count'],
    'store_nbr': 'nunique'
}).round(2)
type_performance.columns = ['Total Sales', 'Avg Sales per Transaction', 'Transaction Count', 'Store Count']
type_performance = type_performance.sort_values('Total Sales', ascending=False)

print(type_performance)

print("\nStore Type Characteristics:")
print(f"  Type A (Premium): 1 store, highest avg sales ({type_performance.loc['A', 'Avg Sales per Transaction']:.2f})")
print(f"  Type B (Large):   1 store")
print(f"  Type C (Medium):  3 stores, lowest performance")
print(f"  Type D (Small):   3 stores, good performance")
print(f"  Type E (Micro):   3 stores")

# Analyze by city
print("\nPerformance by City:")
print("=" * 70)

city_performance = df.groupby('city').agg({
    'unit_sales': ['sum', 'mean', 'count'],
    'store_nbr': 'nunique'
}).round(2)
city_performance.columns = ['Total Sales', 'Avg Sales', 'Transaction Count', 'Store Count']
city_performance = city_performance.sort_values('Total Sales', ascending=False)

print(city_performance)

print(f"\nGuayaquil dominates: {city_performance.loc['Guayaquil', 'Total Sales'] / city_performance['Total Sales'].sum() * 100:.1f}% of total sales")

In [ ]:
# Analyze by cluster
print("\nPerformance by Cluster:")
print("=" * 70)

cluster_performance = df.groupby('cluster').agg({
    'unit_sales': ['sum', 'mean', 'count'],
    'store_nbr': 'nunique'
}).round(2)
cluster_performance.columns = ['Total Sales', 'Avg Sales', 'Transaction Count', 'Store Count']
cluster_performance = cluster_performance.sort_values('Total Sales', ascending=False)

print(cluster_performance)

print("\nCluster Insights:")
for cluster_id in cluster_performance.index:
    stores_in_cluster = df[df['cluster'] == cluster_id]['store_nbr'].unique()
    print(f"  Cluster {cluster_id}: {len(stores_in_cluster)} stores (#{', #'.join(map(str, sorted(stores_in_cluster)))})")

# Summary statistics
print("\n" + "=" * 70)
print("STORE PERFORMANCE SUMMARY")
print("=" * 70)
print(f"Total stores analyzed: 11")
print(f"Total sales in sample: {df['unit_sales'].sum():,.0f} units")
print(f"Performance variation: 4.25x (top vs bottom store)")
print(f"City concentration: 73.8% in Guayaquil")
print(f"Top store: #51 (Type A, Cluster 17)")
print(f"Bottom store: #32 (Type C, Cluster 3)")

In [ ]:
# Visualize store performance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Total sales by store
ax1 = axes[0, 0]
store_sales = df.groupby('store_nbr')['unit_sales'].sum().sort_values(ascending=False)
bars1 = ax1.bar(range(len(store_sales)), store_sales.values, color='steelblue', alpha=0.7, edgecolor='black')
ax1.set_xticks(range(len(store_sales)))
ax1.set_xticklabels([f'#{s}' for s in store_sales.index], rotation=45, ha='right')
ax1.set_ylabel('Total Sales (units)', fontsize=11)
ax1.set_title('Total Sales by Store (11 Guayas Stores)', fontsize=13, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Highlight top/bottom
bars1[0].set_color('#2ca02c')  # Top - green
bars1[-1].set_color('#d62728')  # Bottom - red

# Plot 2: Average sales by store type
ax2 = axes[0, 1]
type_avg = df.groupby('type')['unit_sales'].mean().sort_values(ascending=False)
colors_type = ['#2ca02c', '#ff7f0e', '#9467bd', '#8c564b', '#e377c2']
bars2 = ax2.bar(type_avg.index, type_avg.values, color=colors_type, alpha=0.7, edgecolor='black')
ax2.set_ylabel('Average Sales per Transaction', fontsize=11)
ax2.set_title('Average Sales by Store Type', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}', ha='center', va='bottom', fontsize=10)

# Plot 3: Sales by city
ax3 = axes[1, 0]
city_sales = df.groupby('city')['unit_sales'].sum().sort_values(ascending=False)
wedges, texts, autotexts = ax3.pie(city_sales.values, labels=city_sales.index, autopct='%1.1f%%',
                                     startangle=90, colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
ax3.set_title('Sales Distribution by City', fontsize=13, fontweight='bold')

# Make percentage text more readable
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(10)
    autotext.set_weight('bold')

# Plot 4: Sales by cluster
ax4 = axes[1, 1]
cluster_sales = df.groupby('cluster')['unit_sales'].sum().sort_values(ascending=False)
bars4 = ax4.bar([str(c) for c in cluster_sales.index], cluster_sales.values, 
                color='coral', alpha=0.7, edgecolor='black')
ax4.set_xlabel('Cluster', fontsize=11)
ax4.set_ylabel('Total Sales (units)', fontsize=11)
ax4.set_title('Sales by Store Cluster', fontsize=13, fontweight='bold')
ax4.grid(axis='y', alpha=0.3)

# Add store count labels
for i, (cluster_id, sales) in enumerate(cluster_sales.items()):
    store_count = df[df['cluster'] == cluster_id]['store_nbr'].nunique()
    ax4.text(i, sales, f'{store_count} stores', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(OUTPUTS / '02_store_performance_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nOK - Visualization saved to outputs/figures/eda/02_store_performance_analysis.png")

In [ ]:
# Section 4 summary
print("=" * 70)
print("SECTION 4 COMPLETE - Store-Level Performance Analysis")
print("=" * 70)

print("\nKey Findings:")
print("\n1. Store Performance Hierarchy:")
print(f"   • Top performer: Store #51 (Type A, Cluster 17) - 356,659 units")
print(f"   • Bottom performer: Store #32 (Type C, Cluster 3) - 83,947 units")
print(f"   • Performance gap: 4.25x")

print("\n2. Store Type Patterns:")
print(f"   • Type A (Premium): Highest avg sales (9.63 units/transaction)")
print(f"   • Type C (Medium): Lowest performance (4.78 units/transaction)")
print(f"   • Types D & E: Similar performance (~6.5-6.9 avg)")

print("\n3. Geographic Concentration:")
print(f"   • Guayaquil: 73.8% of total sales (8 stores)")
print(f"   • Other cities: 26.2% combined (Daule, Libertad, Playas)")

print("\n4. Cluster Analysis:")
print(f"   • Cluster 10: Largest volume (4 stores, 664K units)")
print(f"   • Cluster 17: Highest efficiency (1 store, 9.63 avg)")
print(f"   • Cluster 3: Needs investigation (3 stores, lowest avg)")

print("\nBusiness Implications:")
print("   → Store #51 best practices should be studied and replicated")
print("   → Cluster 3 stores (30, 32, 35) require performance improvement plans")
print("   → Type A stores justify premium positioning with 2x avg sales vs Type C")

print("\nTime elapsed: ~1.5 hours / 5.5 hours allocated")
print("Status: Ahead of schedule ✓")

## 5. Item Coverage Analysis

**Objective:** Understand product availability patterns - which items sell in which stores

**Activities:**
- Create store-item availability matrix
- Calculate coverage per store (% of 2,296 items sold)
- Identify universal items (sold in all stores) vs niche items (few stores)
- Analyze coverage by product family
- Zero-sales preliminary analysis

**Expected output:** 
- Item coverage matrix
- Universal vs niche items classification
- Coverage report by store and family

In [ ]:
# Create store-item coverage matrix
print("Item Coverage Analysis")
print("=" * 70)

# Count unique items per store
items_per_store = df.groupby('store_nbr')['item_nbr'].nunique().sort_values(ascending=False)

print(f"Total unique items in sample: {df['item_nbr'].nunique():,}")
print(f"\nUnique items sold per store:")
for store_nbr, item_count in items_per_store.items():
    coverage_pct = item_count / df['item_nbr'].nunique() * 100
    store_info = df[df['store_nbr'] == store_nbr][['type', 'city']].iloc[0]
    print(f"  Store #{store_nbr:>2} (Type {store_info['type']}, {store_info['city']:<12}): {item_count:>4,} items ({coverage_pct:>5.1f}%)")

print(f"\nCoverage statistics:")
print(f"  Max coverage: {items_per_store.max():,} items ({items_per_store.max()/df['item_nbr'].nunique()*100:.1f}%)")
print(f"  Min coverage: {items_per_store.min():,} items ({items_per_store.min()/df['item_nbr'].nunique()*100:.1f}%)")
print(f"  Avg coverage: {items_per_store.mean():.0f} items ({items_per_store.mean()/df['item_nbr'].nunique()*100:.1f}%)")

In [ ]:
# Analyze item distribution across stores
print("\nItem Distribution Across Stores:")
print("=" * 70)

# Count how many stores each item appears in
stores_per_item = df.groupby('item_nbr')['store_nbr'].nunique()

print(f"\nItems by store count:")
print(f"  Sold in all 11 stores (universal):  {(stores_per_item == 11).sum():>5,} items ({(stores_per_item == 11).sum()/len(stores_per_item)*100:>5.1f}%)")
print(f"  Sold in 8-10 stores (widespread):   {((stores_per_item >= 8) & (stores_per_item < 11)).sum():>5,} items ({((stores_per_item >= 8) & (stores_per_item < 11)).sum()/len(stores_per_item)*100:>5.1f}%)")
print(f"  Sold in 5-7 stores (common):        {((stores_per_item >= 5) & (stores_per_item < 8)).sum():>5,} items ({((stores_per_item >= 5) & (stores_per_item < 8)).sum()/len(stores_per_item)*100:>5.1f}%)")
print(f"  Sold in 2-4 stores (selective):     {((stores_per_item >= 2) & (stores_per_item < 5)).sum():>5,} items ({((stores_per_item >= 2) & (stores_per_item < 5)).sum()/len(stores_per_item)*100:>5.1f}%)")
print(f"  Sold in only 1 store (niche):       {(stores_per_item == 1).sum():>5,} items ({(stores_per_item == 1).sum()/len(stores_per_item)*100:>5.1f}%)")

print(f"\nDistribution statistics:")
print(f"  Mean stores per item: {stores_per_item.mean():.1f}")
print(f"  Median stores per item: {stores_per_item.median():.0f}")

# Identify universal and niche items
universal_items = stores_per_item[stores_per_item == 11].index.tolist()
niche_items = stores_per_item[stores_per_item == 1].index.tolist()

print(f"\nUniversal items (sold in all 11 stores): {len(universal_items):,}")
print(f"Niche items (sold in only 1 store): {len(niche_items):,}")

# Check which families have universal items
if len(universal_items) > 0:
    universal_family_dist = df[df['item_nbr'].isin(universal_items)]['family'].value_counts()
    print(f"\nUniversal items by family:")
    print(universal_family_dist)

In [ ]:
# Analyze coverage by product family
print("\nCoverage by Product Family:")
print("=" * 70)

family_coverage = df.groupby(['family', 'store_nbr'])['item_nbr'].nunique().unstack(fill_value=0)

print("\nItems per family per store:")
print(family_coverage)

# Summary by family
family_summary = pd.DataFrame({
    'Total Items': df.groupby('family')['item_nbr'].nunique(),
    'Avg Items per Store': family_coverage.mean(axis=1).round(0),
    'Min Coverage': family_coverage.min(axis=1),
    'Max Coverage': family_coverage.max(axis=1),
    'Coverage Range': family_coverage.max(axis=1) - family_coverage.min(axis=1)
})

print("\nFamily Coverage Summary:")
print(family_summary)

print("\nInsights:")
for family in family_summary.index:
    total = family_summary.loc[family, 'Total Items']
    avg = family_summary.loc[family, 'Avg Items per Store']
    coverage_pct = (avg / total * 100)
    print(f"  {family}: {avg:.0f}/{total:.0f} items per store ({coverage_pct:.1f}% avg coverage)")
    

In [ ]:
# Visualize item coverage patterns
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Items per store
ax1 = axes[0, 0]
bars1 = ax1.bar(range(len(items_per_store)), items_per_store.values, 
                color='steelblue', alpha=0.7, edgecolor='black')
ax1.set_xticks(range(len(items_per_store)))
ax1.set_xticklabels([f'#{s}' for s in items_per_store.index], rotation=45, ha='right')
ax1.set_ylabel('Unique Items Sold', fontsize=11)
ax1.set_title('Item Coverage by Store', fontsize=13, fontweight='bold')
ax1.axhline(df['item_nbr'].nunique(), color='red', linestyle='--', alpha=0.5, label='Total Items (2,296)')
ax1.grid(axis='y', alpha=0.3)
ax1.legend()

# Highlight top/bottom
bars1[0].set_color('#2ca02c')  # Top
bars1[-1].set_color('#d62728')  # Bottom

# Plot 2: Distribution of items by store count
ax2 = axes[0, 1]
store_count_bins = [1, 2, 5, 8, 11, 12]
store_count_labels = ['1 store\n(niche)', '2-4 stores\n(selective)', 
                      '5-7 stores\n(common)', '8-10 stores\n(widespread)', 
                      '11 stores\n(universal)']
store_counts = pd.cut(stores_per_item, bins=store_count_bins, labels=store_count_labels, right=False)
store_counts_dist = store_counts.value_counts().sort_index()

colors2 = ['#d62728', '#ff7f0e', '#ffbb78', '#aec7e8', '#2ca02c']
bars2 = ax2.bar(range(len(store_counts_dist)), store_counts_dist.values, 
                color=colors2, alpha=0.7, edgecolor='black')
ax2.set_xticks(range(len(store_counts_dist)))
ax2.set_xticklabels(store_counts_dist.index, rotation=0, ha='center')
ax2.set_ylabel('Number of Items', fontsize=11)
ax2.set_title('Item Distribution by Store Count', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}\n({height/len(stores_per_item)*100:.1f}%)',
             ha='center', va='bottom', fontsize=9)

# Plot 3: Coverage by family and store
ax3 = axes[1, 0]
family_coverage_pct = (family_coverage.T / family_summary['Total Items'] * 100).T
sns.heatmap(family_coverage_pct, annot=True, fmt='.0f', cmap='YlGnBu', 
            cbar_kws={'label': 'Coverage %'}, ax=ax3, vmin=60, vmax=95)
ax3.set_xlabel('Store Number', fontsize=11)
ax3.set_ylabel('Product Family', fontsize=11)
ax3.set_title('Item Coverage % by Family and Store', fontsize=13, fontweight='bold')

# Plot 4: Coverage summary by family
ax4 = axes[1, 1]
x = np.arange(len(family_summary))
width = 0.25

bars_min = ax4.bar(x - width, family_summary['Min Coverage'], width, 
                   label='Min', color='#d62728', alpha=0.7)
bars_avg = ax4.bar(x, family_summary['Avg Items per Store'], width, 
                   label='Avg', color='#2ca02c', alpha=0.7)
bars_max = ax4.bar(x + width, family_summary['Max Coverage'], width, 
                   label='Max', color='#1f77b4', alpha=0.7)

ax4.set_ylabel('Number of Items', fontsize=11)
ax4.set_title('Coverage Range by Product Family', fontsize=13, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(family_summary.index, rotation=0)
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUTS / '03_item_coverage_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nOK - Visualization saved to outputs/figures/eda/03_item_coverage_analysis.png")

In [ ]:
# Section 5 summary
print("=" * 70)
print("SECTION 5 COMPLETE - Item Coverage Analysis")
print("=" * 70)

print("\nKey Findings:")
print("\n1. Overall Coverage:")
print(f"   • Total unique items in sample: 2,296")
print(f"   • Average coverage per store: 1,790 items (78.0%)")
print(f"   • Coverage range: 1,486 to 2,064 items (64.7% to 89.9%)")

print("\n2. Item Distribution Patterns:")
print(f"   • Universal items (all 11 stores): 1,124 (49.0%)")
print(f"   • Widespread items (8-10 stores): 481 (20.9%)")
print(f"   • Common items (5-7 stores): 260 (11.3%)")
print(f"   • Selective items (2-4 stores): 390 (17.0%)")
print(f"   • Niche items (1 store only): 41 (1.8%)")

print("\n3. Store Type Correlation:")
print(f"   • Type A (Store #51): Best coverage (89.9% - 2,064 items)")
print(f"   • Type D stores: Good coverage (81-88%)")
print(f"   • Type C stores: Lowest coverage (65-66%)")

print("\n4. Family Coverage:")
print(f"   • BEVERAGES: 81.0% avg coverage (485/599 items per store)")
print(f"   • CLEANING: 77.1% avg coverage (340/441 items per store)")
print(f"   • GROCERY I: 76.9% avg coverage (966/1,256 items per store)")

print("\nBusiness Implications:")
print("   → Nearly half of items (49%) are universal - strong core assortment")
print("   → Type C stores need assortment expansion (15-25% gap vs top stores)")
print("   → Only 1.8% niche items - minimal fragmentation")
print("   → Mean 8.6 stores per item indicates good distribution efficiency")

print("\nTime elapsed: ~2.0 hours / 5.5 hours allocated")
print("Status: Ahead of schedule ✓")

## 6. Calendar Gap Filling

**Objective:** Create complete daily time series for each store-item pair

**Activities:**
- Convert date to datetime format
- Identify missing dates per store-item group
- Fill calendar gaps with zero sales (complete daily index)
- Validate completeness
- Compare row count before/after

**Expected output:** 
- Complete daily calendar (no gaps)
- Expanded dataset with zero-filled missing dates
- Validation report

In [ ]:
# Convert date to datetime
print("Calendar Gap Filling")
print("=" * 70)

print("Step 1: Convert date to datetime format...")
df['date'] = pd.to_datetime(df['date'])

print(f"  Date dtype: {df['date'].dtype}")
print(f"  Date range: {df['date'].min()} to {df['date'].max()}")
print(f"  Total days: {(df['date'].max() - df['date'].min()).days + 1} days")

# Check current temporal coverage
print("\nStep 2: Analyze temporal coverage...")
print(f"  Current rows: {len(df):,}")
print(f"  Unique dates in data: {df['date'].nunique():,}")
print(f"  Unique store-item pairs: {df.groupby(['store_nbr', 'item_nbr']).ngroups:,}")

# Calculate expected rows (if complete)
expected_rows = (df['date'].max() - df['date'].min()).days + 1
expected_rows *= df.groupby(['store_nbr', 'item_nbr']).ngroups

print(f"\nExpected rows (complete calendar): {expected_rows:,}")
print(f"Current rows: {len(df):,}")
print(f"Gap: {expected_rows - len(df):,} missing date-store-item combinations")

In [ ]:
# Analyze sparsity and make filling decision
print("\nStep 3: Sparsity Analysis...")
print("=" * 70)

sparsity = (1 - len(df) / expected_rows) * 100
print(f"  Data sparsity: {sparsity:.1f}%")
print(f"  Coverage: {(len(df) / expected_rows) * 100:.2f}%")

print("\nBusiness Reality:")
print("  → Most store-item-date combinations have ZERO sales")
print("  → Items not sold on specific dates (not stocked/no demand)")
print("  → Filling ALL gaps would create 33.2M row dataset (110x expansion)")

print("\nDecision Point:")
print("  Option A: Fill ALL gaps → 33M rows (unmanageable for development)")
print("  Option B: Fill gaps only for 'active' items → Still very large")
print("  Option C: Keep sparse format → 300K rows (manageable)")

print("\nRECOMMENDATION: Option C - Keep Sparse Format")
print("  Rationale:")
print("    • Retail data is naturally sparse (items not sold every day)")
print("    • 33M rows exceeds development budget (memory/time)")
print("    • Most forecasting models handle missing dates internally")
print("    • Zero-filling appropriate only for true stockouts (needs investigation)")

print("\nAlternative for modeling phase:")
print("  → Create subset of 'active items' (sold ≥10% of days)")
print("  → Use time series models that handle irregular intervals")
print("  → Document as known limitation in project")

print("\nDECISION (DEC-005): Sparse Data Handling")
print("  Context: 99.1% of store-item-date combinations have no sales")
print("  Decision: Keep sparse format (300K rows), do NOT fill all calendar gaps")
print("  Rationale: Retail reality, memory constraints, modeling flexibility")
print("  Impact: Models must handle irregular time intervals")

In [ ]:
# Validate sparse format decision
print("\nStep 4: Validate Sparse Format...")
print("=" * 70)

# Check date completeness per item (sample analysis)
sample_items = df.groupby('item_nbr')['date'].count().nlargest(10)

print("Top 10 most frequently sold items (days with sales):")
for item_nbr, days_sold in sample_items.items():
    total_days = (df['date'].max() - df['date'].min()).days + 1
    frequency_pct = days_sold / total_days * 100
    family = df[df['item_nbr'] == item_nbr]['family'].iloc[0]
    print(f"  Item #{item_nbr}: {days_sold:>4} days ({frequency_pct:>5.1f}%) - {family}")

print(f"\nMedian sales frequency across all items:")
median_freq = df.groupby('item_nbr')['date'].count().median()
total_days = (df['date'].max() - df['date'].min()).days + 1
print(f"  {median_freq:.0f} days sold out of {total_days} total days ({median_freq/total_days*100:.1f}%)")

print("\nConclusion:")
print("  → Even top items sold only ~25% of days")
print("  → Median item sold <10% of days")
print("  → Sparse format is appropriate")
print("  → Calendar completeness NOT required for this project")

print("\n" + "=" * 70)
print("SECTION 6 COMPLETE - Calendar Analysis (Sparse Format Retained)")
print("=" * 70)

## 7. Date Feature Extraction

**Objective:** Create temporal features for time series analysis

**Activities:**
- Extract year, month, day, day_of_week
- Add is_weekend flag
- Add day_of_month, week_of_year
- Validate feature distributions
- Final dataset summary

**Expected output:** 
- 6+ date-based features
- Clean, analysis-ready dataset
- Day 3 completion summary

In [ ]:
# Extract date features
print("Date Feature Extraction")
print("=" * 70)

print("Creating temporal features...")

# Basic date components (some already exist from outlier detection)
if 'year' not in df.columns:
    df['year'] = df['date'].dt.year
if 'month' not in df.columns:
    df['month'] = df['date'].dt.month
if 'day_of_week' not in df.columns:
    df['day_of_week'] = df['date'].dt.dayofweek

# Additional features
df['day'] = df['date'].dt.day
df['day_of_month'] = df['date'].dt.day
df['week_of_year'] = df['date'].dt.isocalendar().week
df['quarter'] = df['date'].dt.quarter
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)  # Saturday=5, Sunday=6
df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)

print(f"OK - Created {10} temporal features")

print("\nTemporal features created:")
temporal_features = ['year', 'month', 'day', 'day_of_week', 'day_of_month', 
                     'week_of_year', 'quarter', 'is_weekend', 'is_month_start', 'is_month_end']
for feat in temporal_features:
    print(f"  • {feat}")

print(f"\nCurrent dataset shape: {df.shape}")
print(f"Total columns: {len(df.columns)}")

In [ ]:
# Validate temporal feature distributions
print("\nTemporal Feature Validation:")
print("=" * 70)

print("\nYear distribution:")
print(df['year'].value_counts().sort_index())

print("\nMonth distribution:")
print(df['month'].value_counts().sort_index())

print("\nDay of week distribution (0=Monday, 6=Sunday):")
dow_counts = df['day_of_week'].value_counts().sort_index()
dow_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for dow, count in dow_counts.items():
    print(f"  {dow} ({dow_names[dow]}): {count:>7,} ({count/len(df)*100:>5.2f}%)")

print("\nQuarter distribution:")
print(df['quarter'].value_counts().sort_index())

print("\nWeekend vs Weekday:")
print(f"  Weekday: {(df['is_weekend'] == 0).sum():>7,} ({(df['is_weekend'] == 0).sum()/len(df)*100:>5.2f}%)")
print(f"  Weekend: {(df['is_weekend'] == 1).sum():>7,} ({(df['is_weekend'] == 1).sum()/len(df)*100:>5.2f}%)")

print("\nMonth start/end:")
print(f"  Month start: {df['is_month_start'].sum():>6,} ({df['is_month_start'].sum()/len(df)*100:>5.2f}%)")
print(f"  Month end:   {df['is_month_end'].sum():>6,} ({df['is_month_end'].sum()/len(df)*100:>5.2f}%)")

print("\nOK - All temporal features validated")

In [ ]:
# Final dataset summary
print("=" * 70)
print("DAY 3 COMPLETE - Data Quality & Store Analysis")
print("=" * 70)

print("\nFinal Dataset Summary:")
print(f"  Rows: {len(df):,}")
print(f"  Columns: {len(df.columns)}")
print(f"  Memory: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print("\nColumn Categories:")
print(f"  Core data: id, date, store_nbr, item_nbr, unit_sales, onpromotion")
print(f"  Product metadata: family, class, perishable")
print(f"  Store metadata: city, state, type, cluster")
print(f"  Outlier flags: outlier_iqr, outlier_zscore, outlier_forest, outlier_all3, outlier_2plus, outlier_any")
print(f"  Temporal features: year, month, day, day_of_week, quarter, is_weekend, etc. (10 features)")

print("\nData Quality Status:")
print(f"  Missing values: {df.isnull().sum().sum()} (0%)")
print(f"  Negative sales: 0 (already handled)")
print(f"  Outliers flagged: {df['outlier_all3'].sum():,} high-confidence (0.28%)")
print(f"  Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"  Sparse format: 0.9% coverage (99.1% sparsity - normal for retail)")

print("\nKey Accomplishments:")
print("  ✓ Missing values handled (onpromotion filled)")
print("  ✓ Three-method outlier detection (IQR + Z-score + Isolation Forest)")
print("  ✓ Store performance analyzed (11 stores, 4.25x variation)")
print("  ✓ Item coverage mapped (49% universal items)")
print("  ✓ Sparsity documented (retail reality)")
print("  ✓ Temporal features created (10 features)")

print("\nDecisions Logged:")
print("  DEC-003: Fill onpromotion NaN with False")
print("  DEC-004: Three-method outlier detection, flag but retain")
print("  DEC-005: Keep sparse format (not fill all calendar gaps)")

print("\nReady for Day 4:")
print("  → Feature engineering (rolling averages, lags)")
print("  → Temporal pattern analysis (seasonality, trends)")
print("  → Product dynamics (fast/slow movers)")

print(f"\n{'='*70}")
print(f"Time spent: ~4.5 hours / 5.5 hours allocated")
print(f"Status: 1 hour under budget! ✓")
print(f"{'='*70}")

print("\nNext: Save notebook, commit to Git, start Day 4")